In [1]:
%load_ext autoreload
%autoreload 2
import os

## Activate mriqc

Firstly, we need to activate fmriprep. This will add the relevant files to our path. This only needs doing once (you dont have to do it again next time you login).



In [2]:
from CINN_fmriprep.fmriprep import activate_mriqc
activate_mriqc()

Activated


## Define an mriqc handler.

Here I define an mriqchandler. The main arguments are the path to the BIDS directory on which to do the mriqcing, the output directory and the work directory for intermediate files. slurmout_path will define where the files that update on progress and errors will be stored.  

In this case, I also define a subject on whom to do the mriqcing. If this argument is not included it defaults to doing all subjects and then combining the results into a group report: https://mriqc.readthedocs.io/en/stable/running.html#running-mriqc-on-hpc-clusters:~:text=Possible%20choices%3A%20participant%2C%20group

In [3]:
from CINN_fmriprep.fmriprep import MriqcHandler as mriqc
k= mriqc('/storage/basic/nh_leverhulme/Richardson_2018','/storage/basic/nh_leverhulme/mriqc','/storage/basic/nh_leverhulme/work2',
        slurmout_path='/home/users/yg916972/Scripts/',subject='sub-pixar002')


I now make the SLURM job for the subject using the make_slurm method.

In [4]:
k.make_slurm()

Now we have an mriqc template script created. 

In [5]:
print(k.slurm)

#!/bin/sh

#SBATCH --cpus-per-task       8
#SBATCH --error               /home/users/yg916972/Scripts/mriqc_sub-pixar002.err
#SBATCH --mail-type           ALL
#SBATCH --mail-user           n.hedger@reading.ac.uk
#SBATCH --mem-per-cpu         2G
#SBATCH --nodes               1-1
#SBATCH --output              /home/users/yg916972/Scripts/mriqc_sub-pixar002.out

source ~/.bashrc
cd /storage



Upon being submitted, the following command will be submitted to the job queue using sbatch

In [6]:
k.cmd

'mriqc /storage/basic/nh_leverhulme/Richardson_2018 /storage/basic/nh_leverhulme/mriqc/sub-pixar002 participant --participant-label sub-pixar002 -w /storage/basic/nh_leverhulme/work2/sub-pixar002 '

If I wanted to add some additional arguments or output spaces to mriqc, I could do so as follows

In [7]:
k.make_slurm(additionals=['--verbose-reports'])

In [8]:
k.cmd

'mriqc /storage/basic/nh_leverhulme/Richardson_2018 /storage/basic/nh_leverhulme/mriqc/sub-pixar002 participant --participant-label sub-pixar002 -w /storage/basic/nh_leverhulme/work2/sub-pixar002 --verbose-reports'

The SLURM/ SBATCH pre-commands for provisioning resources are sensible defaults, but If I wanted to add in more, or different slurm pre-commands, I could do this by adding these to the 'slurm' subdictionary of the yml file that comes with the package.

In [9]:
k.y['mriqc_slurm']

{'cpus_per_task': 8,
 'mem_per_cpu': '2G',
 'N': '1-1',
 'mail_user': 'n.hedger@reading.ac.uk',
 'mail_type': 'ALL'}

In other words, I would modify this file:

In [10]:
k.yaml

'/home/users/yg916972/Software/CINN_fmriprep/config/config.yml'

The job can be submitted as follows. A message is printed to point to the location of the .out and .err files, which will contain outputs and errors respectively.

In [11]:
k.submit_slurm()

Submitted batch job 95107

Job with mriqc_sub-pixar002 submitted! 

               Outputs will be found at /storage/basic/nh_leverhulme/mriqc/sub-pixar002 
 
              Progress updated in /home/users/yg916972/Scripts/mriqc_sub-pixar002.out

              Errors will be reported at /home/users/yg916972/Scripts/mriqc_sub-pixar002.err



## Submitting multiple jobs in parallel. 

What we may often want to do is submit multiple subjects as multiple jobs to run in parallel. This process is mostly the same as before, but this time we can use the MultipleMriqcHandler. Here, we specify the additional argument of 'subjects' which is passed as a list of strings.

In [20]:
from CINN_fmriprep.fmriprep import MultipleMriqcHandler as MMQH

In [13]:
k= MMQH('/storage/basic/nh_leverhulme/Richardson_2018','/storage/basic/nh_leverhulme/mriqc','/storage/basic/nh_leverhulme/work2',
        slurmout_path='/home/users/yg916972/Scripts/',subjects=['sub-pixar032','sub-pixar033'])

In [14]:
k.make_mriqc_handlers()
k.make_slurms()

Now we have a list of FmriPrepHandlers (one for each subject) which correspond to different jobscripts and commands. 

In [15]:
print(k.handlers[0].slurm)

#!/bin/sh

#SBATCH --cpus-per-task       8
#SBATCH --error               /home/users/yg916972/Scripts/mriqc_sub-pixar032.err
#SBATCH --mail-type           ALL
#SBATCH --mem-per-cpu         2G
#SBATCH --nodes               1-1
#SBATCH --output              /home/users/yg916972/Scripts/mriqc_sub-pixar032.out

source ~/.bashrc
cd /storage



In [16]:
k.handlers[0].cmd

'mriqc /storage/basic/nh_leverhulme/Richardson_2018 /storage/basic/nh_leverhulme/mriqc/sub-pixar032 participant --participant-label sub-pixar032 -w /storage/basic/nh_leverhulme/work2/sub-pixar032 '

In [17]:
print(k.handlers[1].slurm)

#!/bin/sh

#SBATCH --cpus-per-task       8
#SBATCH --error               /home/users/yg916972/Scripts/mriqc_sub-pixar033.err
#SBATCH --mail-type           ALL
#SBATCH --mem-per-cpu         2G
#SBATCH --nodes               1-1
#SBATCH --output              /home/users/yg916972/Scripts/mriqc_sub-pixar033.out

source ~/.bashrc
cd /storage



In [18]:
print(k.handlers[1].cmd)

mriqc /storage/basic/nh_leverhulme/Richardson_2018 /storage/basic/nh_leverhulme/mriqc/sub-pixar033 participant --participant-label sub-pixar033 -w /storage/basic/nh_leverhulme/work2/sub-pixar033 


In [19]:
k.submit_slurms()

Submitted batch job 95109

Job with mriqc_sub-pixar032 submitted! 

               Outputs will be found at /storage/basic/nh_leverhulme/mriqc/sub-pixar032 
 
              Progress updated in /home/users/yg916972/Scripts/mriqc_sub-pixar032.out

              Errors will be reported at /home/users/yg916972/Scripts/mriqc_sub-pixar032.err

Submitted batch job 95110

Job with mriqc_sub-pixar033 submitted! 

               Outputs will be found at /storage/basic/nh_leverhulme/mriqc/sub-pixar033 
 
              Progress updated in /home/users/yg916972/Scripts/mriqc_sub-pixar033.out

              Errors will be reported at /home/users/yg916972/Scripts/mriqc_sub-pixar033.err

